In [1]:
#required libraries/packages
!pip install open_clip_torch
!pip install requests
!pip install clip
!pip install faiss-cpu
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-0.2.0-py3-none-any.whl size=6989 sha256=20cf1e590b8399bf52d3de83186b34ff8e581217c9685ad925795ecb00ca1b00
  Stored in directory: /root/.cache/pip/wheels/6c/fd/54/9d4e15cf829b871199a7cd3597e869a514d1624a0a43076896
Successfully built clip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 2.8 MB/s eta 0:00:00


In [2]:
#importing required libraries/packages
from google.colab import userdata
import os
from PIL import Image
import torch
# import clip # Remove this line
from transformers import CLIPProcessor, CLIPModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import faiss
from groq import Groq

In [3]:
groq_api_key = userdata.get('GROQ_API_KEY')
os.environ["GROQ_API_KEY"] = groq_api_key

# Initialize CLIP model and processor
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Initialize GROQ API client
groq_client = Groq(api_key=groq_api_key)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [24]:
# Function to extract CLIP embeddings from images
def extract_clip_embeddings(image_paths):
    embeddings = []
    for image_path in image_paths:
        image = Image.open(image_path)
        inputs = clip_processor(images=image, return_tensors="pt")
        outputs = clip_model.get_image_features(**inputs)
        embeddings.append(outputs.detach().numpy()[0])
    return np.array(embeddings)

# Load images and extract embeddings
image_folder = "bankimages"
image_paths = [os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith(".jpg") or file.endswith(".png")]
embeddings = extract_clip_embeddings(image_paths)

# Create FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# Function to search for similar images
def search_similar_images(query_embedding, k=5):
    D, I = index.search(np.array([query_embedding]), k=k)
    return I[0]

# Function to generate answer using GROQ API
def generate_answer(query, context):
    response = groq_client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": f"Query: {query}\nContext: {context}"},
        ],
    )
    return response.choices[0].message.content

# Main function for Q&A
def image_qa(query):
    # Tokenize query and get CLIP embedding
    # query_inputs = clip.tokenize([query]) # Remove this line
    # query_outputs = clip_model.get_text_features(**query_inputs) # Remove this line
    # query_embedding = query_outputs.detach().numpy()[0] # Remove this line
    inputs = clip_processor(text=[query], return_tensors="pt", padding=True, truncation=True) # Add this line
    text_features = clip_model.get_text_features(**inputs) # Add this line
    query_embedding = text_features.detach().numpy()[0] # Add this line


    # Search for similar images
    similar_image_indices = search_similar_images(query_embedding)

    # Get context from similar images
    context = ""
    for index in similar_image_indices:
        context += f"Image {index}: {image_paths[index]}\n"

    # Generate answer using GROQ API
    answer = generate_answer(query, context)
    return answer


In [13]:
!pip install groq langchain-groq

In [17]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 6.1 MB/s eta 0:00:00


In [14]:
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq

In [20]:
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
from langchain_core.embeddings import Embeddings # Import Embeddings

llm = ChatGroq(temperature=0, model_name="llama3-8b-8192")

# Define a custom embedding function using CLIP
class CLIPEmbeddings(Embeddings):
    def __init__(self, clip_model, clip_processor):
        self.clip_model = clip_model
        self.clip_processor = clip_processor

    def embed_documents(self, texts):
        # This method is usually for embedding lists of documents,
        # but for image search, we'll primarily use embed_query
        raise NotImplementedError

    def embed_query(self, text):
        inputs = self.clip_processor(text=[text], return_tensors="pt", padding=True, truncation=True)
        text_features = self.clip_model.get_text_features(**inputs)
        return text_features.detach().numpy()[0].tolist() # Return as list of floats

# Create an instance of the custom embedding function
clip_embeddings = CLIPEmbeddings(clip_model, clip_processor)

# Create a FAISS vector store from the embeddings
# Provide the custom embedding function to FAISS.from_embeddings
vectorstore = FAISS.from_embeddings(text_embeddings=list(zip(image_paths, embeddings)), embedding=clip_embeddings)

# Create a retriever from the vector store
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# Create the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

In [22]:
question ="how to do NEFT Fund Transfer"
response = qa_chain({"query": question})
print(response["result"])

Based on the provided images, it appears that the bank's online platform allows for NEFT fund transfers. Here's a step-by-step guide on how to do it:

1. Log in to your online banking account (refer to the "Login-Step1.jpg" and "Login-Step2.jpg" images).
2. Click on the "Fund Transfer" option (refer to the "FundTransfer.jpg" image).
3. Select "NEFT" as the transfer mode.
4. Enter the recipient's account details, including their account number, IFSC code, and name.
5. Enter the transfer amount and select the account from which the funds will be debited.
6. Review the transaction details and confirm the transfer.

Please note that the exact steps may vary depending on the bank's specific online platform and your account settings. It's always a good idea to check with your bank for any specific requirements or restrictions on NEFT transfers.


In [25]:
result = image_qa("how to do NEFT Fund Transfer")
print(result)

I can help you with that!

NEFT (National Electronic Funds Transfer) is an electronic funds transfer system that allows you to transfer funds from one bank account to another. Here's a step-by-step guide on how to do a NEFT fund transfer:

**Step 1: Login to your bank account**

* Go to your bank's website or mobile app.
* Enter your username and password to login to your account.
* You will be redirected to your account dashboard.

(Refer to Image 6: bankimages/Login-Step1.jpg)

**Step 2: Click on Funds Transfer**

* Click on the "Funds Transfer" or "Transfer Funds" option from the dashboard.
* You will be redirected to the funds transfer page.

(Refer to Image 8: bankimages/FundTransfer.jpg)

**Step 3: Select Fund Transfer Mode**

* Select "NEFT" as the mode of fund transfer.
* Enter the account details of the beneficiary (the person you want to transfer funds to).
* Enter the amount you want to transfer.
* Select the account from which you want to transfer funds.

**Step 4: Confirm 

In [4]:
!pip install gradio

In [9]:
import gradio as gr

def clear_inputs():
    return "", ""

def gradio_image_qa(query):
    return image_qa(query)

In [10]:
with gr.Blocks() as iface:
    with gr.Row():
       gr.Markdown("<h1 style='text-align: center; background-color: #f0f0f0;'>FAQs</h1>") # Centered heading with background color
    with gr.Row():
        text_input = gr.Textbox(lines=2, placeholder="Enter your FAQ here", label="FAQ Text")
        output = gr.Textbox(label="Steps")
    with gr.Row():
        submit_button = gr.Button("Submit")
        clear_button = gr.Button("Clear")

    submit_button.click(fn=gradio_image_qa, inputs=[text_input], outputs=output)
    clear_button.click(fn=clear_inputs, inputs=None, outputs=[text_input, output])


iface.launch()

#iface = gr.Interface(fn=gradio_image_qa, inputs="text", outputs="text")
#iface.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8e06e87850acb8b04d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
